In [ ]:
import gc
import random

from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, auc

import h5py
from concurrent.futures import ThreadPoolExecutor

import cv2
import numpy as np
import pandas as pd
import pydicom as dicom
from keras import Model, Input
from keras.src.layers import Dense, Dropout
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.src.callbacks import Callback
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow.python.framework import constant_op
from tensorflow.python.ops import clip_ops, math_ops
from tensorflow.keras import backend as K
from keras.src import ops
from tqdm import tqdm

### ファイル名の定義

In [ ]:
train_image_hdf5 = "train-image.hdf5"
train_metadata_csv = "train-metadata.csv"
test_image_hdf5 = "test-image.hdf5"
test_metadata_csv = "test-metadata.csv"
sample_submission_csv = "sample_submission.csv"

base_path = "/home/toru/PycharmProjects/isic-2024-challenge"

### パラメータの定義

In [ ]:
CONFIG = {
    "SEED": 42,
    "N_FOLDS": 5,
    "N_EPOCHS": 100,
    "BATCH_SIZE": 32,
    "VAL_BATCH_SIZE": 50,
    "LR": 0.001,
    "IMAGE_HEIGHT": 224,
    "IMAGE_WIDTH": 224,
    "IMAGE_CHANNEL": 3,
    "N_CLASSES": 1,
    "PATIENCE": 2,
    "TRAIN_RATIO": 0.8,
    "VAL_RATIO": 0.1,
    "TEST_RATIO": 0.1,
}

In [ ]:
df_train = pd.read_csv(f"{base_path}/{train_metadata_csv}")
df_test = pd.read_csv(f"{base_path}/{test_metadata_csv}")

In [ ]:
# patient_id が unique になるようにデータを抽出
df_positive = df_train[df_train["target"] == 1].reset_index(drop=True)
df_negative = df_train[df_train["target"] == 0].reset_index(drop=True)

df_train = pd.concat([df_positive, df_negative.iloc[:df_positive.shape[0]*20, :]])  # positive:negative = 1:20

In [ ]:
df_train

In [ ]:
train_isic_ids = df_train["isic_id"].values

train_isic_ids_positive = df_train[df_train["target"] == 1]["isic_id"].values
train_isic_ids_negative = df_train[df_train["target"] == 0]["isic_id"].values

In [ ]:
# ラベルが1のデータ数
sum_of_positive = len(df_train[df_train["target"] == 1])

# ラベルが0のデータ数
sum_of_negative = len(df_train[df_train["target"] == 0])

# ラベルが1のデータ数 / ラベルが0のデータ数
positive_negative_ratio = sum_of_positive / sum_of_negative

print(f"positive: {sum_of_positive}")
print(f"negative: {sum_of_negative}")
print(f"positive/negative: {positive_negative_ratio}")

In [ ]:
def get_image_and_label(hdf, isic_id):
    # 画像を取得
    image_data = hdf[isic_id][()]

    # Convert the binary data to a numpy array
    image_data = np.frombuffer(image_data, np.uint8)

    # Decode the image from the numpy array
    image_data = cv2.cvtColor(cv2.imdecode(image_data, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

    image_data = cv2.resize(image_data, (CONFIG["IMAGE_HEIGHT"], CONFIG["IMAGE_WIDTH"]))

    # 画像の正規化 (0 ~ 1) に変換
    image_data = (image_data - image_data.min()) / (image_data.max() - image_data.min())
    
    # クラスを取得する. これが学習時のラベルになる
    label = df_train[df_train["isic_id"] == isic_id]["target"].values[0]
    return image_data, label


def normalize_image(image):
    # 画像の正規化 (0 ~ 1) に変換
    image = (image - image.min()) / (image.max() - image.min())
    return image

def augmentation(image):
    
    # 確率で画像をズーム
    if np.random.rand() > 0.5:
        scale = np.random.uniform(0.8, 1.2)
        image = cv2.resize(image, (0, 0), fx=scale, fy=scale)
        image = cv2.resize(image, (CONFIG["IMAGE_HEIGHT"], CONFIG["IMAGE_WIDTH"]))
    
    # 明るさを変える
    alpha = 1.0 + np.random.uniform(-0.01, 0.01)
    beta = np.random.uniform(-0.01, 0.01)
    image = image * alpha + beta
    
    # 画像をぼかす
    k_size = np.random.randint(1, 10) * 2 + 1
    image = cv2.GaussianBlur(image, (k_size, k_size), 0)

    # 確率で付与するノイズを変える
    if np.random.rand() > 0.5:
        # ガウシアンノイズ
        image = image + np.random.normal(0, 0.1, image.shape)
        image = np.clip(image, 0, 1)
        
    # 画像を確率で反転
    if np.random.rand() > 0.5:
        image = cv2.flip(image, 1)

    # 特定の範囲を切り抜く
    n_cut = np.random.randint(1, 5)
    for _ in range(n_cut):
        WINDOW_SIZE = np.random.randint(10, 50)
        x_min = np.random.randint(0, image.shape[0] - WINDOW_SIZE)
        x_max = x_min + WINDOW_SIZE
        y_min = np.random.randint(0, image.shape[1] - WINDOW_SIZE)
        y_max = y_min + WINDOW_SIZE
        image[x_min:x_max, y_min:y_max] = 0

    # 画像を回転
    angle = np.random.randint(0, 360)
    image = cv2.warpAffine(image, cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1.0),
                           (image.shape[1], image.shape[0]))

    return image

def generator(isic_ids,
              batch_size=CONFIG["BATCH_SIZE"],
              mode="train",
              is_augmentation=False,
              is_shuffle=False,
              is_one_epoch=False,
              is_multi_threading=False
              ):
    hdf_path = None
    if mode == "train":
        hdf_path = f"{base_path}/{train_image_hdf5}"
    elif mode == "test":
        hdf_path = f"{base_path}/{test_image_hdf5}"
    else:
        raise ValueError("mode must be 'train' or 'test'")
    
    def process_image_label(isic_id):
        with h5py.File(hdf_path, "r") as hdf:
            image, label = get_image_and_label(hdf, isic_id)
            if is_augmentation:
                image = augmentation(image)
                
            # 画像を正規化
            image = normalize_image(image)
        return image, label

    while True:
        if is_shuffle:
            random.shuffle(isic_ids)

        for i in range(0, len(isic_ids), batch_size):
            end = min(i + batch_size, len(isic_ids))
            batch_isic_ids = isic_ids[i:end]
            if is_multi_threading:
                with ThreadPoolExecutor() as executor:
                    results = list(executor.map(process_image_label, batch_isic_ids))
            else:
                results = [process_image_label(isic_id) for isic_id in batch_isic_ids]

            images, labels = zip(*results)
            images = np.array(images)
            labels = np.array(labels)

            yield images, labels

        if is_one_epoch:
            break

### データセットの作成

In [ ]:
# id をシャッフル
random.shuffle(train_isic_ids)

# データセットを分割 (positive)
train_datasets_pos = train_isic_ids_positive[:int(len(train_isic_ids) * CONFIG["TRAIN_RATIO"])]
valid_datasets_pos = train_isic_ids_positive[int(len(train_isic_ids) * CONFIG["TRAIN_RATIO"]):]
test_datasets_pos = train_isic_ids_positive[int(len(train_isic_ids) * CONFIG["TRAIN_RATIO"]):]

# データセットを分割 (negative)
train_datasets_neg = train_isic_ids_negative[:int(len(train_isic_ids) * CONFIG["TRAIN_RATIO"])]
valid_datasets_neg = train_isic_ids_negative[int(len(train_isic_ids) * CONFIG["TRAIN_RATIO"]):]
test_datasets_neg = train_isic_ids_negative[int(len(train_isic_ids) * CONFIG["TRAIN_RATIO"]):]

# データセットを結合
train_datasets = np.concatenate([train_datasets_pos, train_datasets_neg])
valid_datasets = np.concatenate([valid_datasets_pos, valid_datasets_neg])
test_datasets = np.concatenate([test_datasets_pos, test_datasets_neg])

# 陽性と陰性のデータ数を表示
print(f"train positive: {len(train_datasets_pos)}, negative: {len(train_datasets_neg)}")

# データセットの数をそれぞれ表示
print(f"train: {len(train_datasets)}, valid: {len(valid_datasets)}, test: {len(test_datasets)}")

In [ ]:
# 画像を1枚だけ表示
isic_id = "ISIC_0015670"
image, label = get_image_and_label(h5py.File(f"{base_path}/{train_image_hdf5}", "r"), isic_id)

plt.imshow(image)
plt.title(f"label: {label}")
plt.show()

In [ ]:
show_img_size = 4
# 画像を取得して表示
plt.figure(figsize=(20, 5))
for images, labels in generator(train_isic_ids, batch_size=show_img_size, mode="train", is_shuffle=False,
                                is_one_epoch=True, is_augmentation=True):
    # 画像の情報を表示
    print(f"image shape: {images.shape}")
    print(f'image type: {images.dtype}')
    print(f"image max: {images.max()}, min: {images.min()}")
    print(f"label shape: {labels.shape}")

    for i, (image, label) in enumerate(zip(images, labels)):
        # 最小値を0, 最大値を1にする
        
        plot_size = min(show_img_size, len(images))
        plt.subplot(1, plot_size, i + 1)
        plt.imshow(image)
        if label == 1:
            plt.title(f"positive")
        else:
            plt.title("negative")
    plt.show()

    break

### モデルの定義

In [ ]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float = 0.80):
    v_gt = abs(np.asarray(solution.values) - 1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1 - min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr ** 2 + (max_fpr - 0.5 * max_fpr ** 2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc


def auroc(y_true, y_pred, min_fpr=0.8):
    v_gt = tf.abs(y_true - 1)
    v_pr = tf.abs(y_pred - 1)
    partial_auc_scaled = tf.py_function(comp_score, [v_gt, v_pr], tf.float64)
    partial_auc = 0.5 * min_fpr ** 2 + (min_fpr - 0.5 * min_fpr ** 2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc


class AUCROCMetric(tf.keras.metrics.Metric):
    def __init__(self, name='aucroc', **kwargs):
        super(AUCROCMetric, self).__init__(name=name, **kwargs)
        self.auc_metric = tf.keras.metrics.AUC()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.auc_metric.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return self.auc_metric.result()

    def reset_states(self):
        self.auc_metric.reset_states()

def binary_crossentropy_balance(target, output):
    # target を output の型にキャスト
    target = tf.dtypes.cast(target, output.dtype)
    epsilon_ = constant_op.constant(0.00001, output.dtype)

    # nan を防ぐためにクリップ
    output = clip_ops.clip_by_value(output, epsilon_, 0.99999)

    # 交差エントロピーの計算
    bce = target * math_ops.log(output + epsilon_) * 3.0
    bce += (1.0 - target) * math_ops.log(1.0 - output + epsilon_)

    bce_sum = -K.sum(bce, axis=-1)
    return bce_sum

def create_model(model_name='DenseNet201'):
    # カスタム入力層
    input_shape = (CONFIG["IMAGE_HEIGHT"], CONFIG["IMAGE_WIDTH"], CONFIG["IMAGE_CHANNEL"])
    inputs = Input(shape=input_shape)

    x = inputs

    # DenseNet201
    base_model = tf.keras.applications.DenseNet201(
        include_top=True,
        weights="imagenet",
        input_tensor=x,
        input_shape=input_shape,
        pooling="avg",
        classes=1000,
        classifier_activation='softmax',
    )

    # ベースモデルの出力
    x = base_model.output
    x = Dense(1000, activation='sigmoid')(x)
    x = Dense(1, activation='sigmoid')(x)

    # モデルの作成
    model = Model(inputs=inputs, outputs=x, name=model_name)

    # optimizer : Adam
    opt = tf.keras.optimizers.AdamW(learning_rate=CONFIG["LR"])

    # モデルのコンパイル
    model.compile(optimizer=opt, loss=binary_crossentropy_balance, metrics=['accuracy'])

    return model

In [ ]:
model = create_model()
# モデルの概要を表示
# model.summary()

In [ ]:
from keras.src.callbacks import EarlyStopping


class CustomCheckpoint(Callback):
    def __init__(self, filepath):
        super(CustomCheckpoint, self).__init__()
        self.filepath = filepath
        self.best_val_accuracy = 1.0

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_log_loss_metrics = logs.get('val_loss')

        if val_log_loss_metrics is not None and val_log_loss_metrics < self.best_val_accuracy and val_log_loss_metrics < 0.8:
            self.best_val_accuracy = val_log_loss_metrics
            filepath = f'{self.filepath}.keras'
            self.model.save(filepath, overwrite=True)
            print(f'Saved model to {filepath} with validation accuracy: {val_log_loss_metrics:.4f}')


# カスタムチェックポイントのコールバックを作成
custom_checkpoint = CustomCheckpoint(filepath='model_dense')
Ecall = EarlyStopping(monitor='val_loss', patience=CONFIG["PATIENCE"], restore_best_weights=True)

In [ ]:
history = model.fit(
    generator(train_datasets, is_augmentation=True, is_shuffle=True),
    steps_per_epoch=len(train_datasets) // CONFIG["BATCH_SIZE"],
    epochs=CONFIG["N_EPOCHS"],
    validation_data=generator(valid_datasets, is_augmentation=False, is_shuffle=False),
    validation_steps=len(valid_datasets) // CONFIG["VAL_BATCH_SIZE"],
    callbacks=[custom_checkpoint, Ecall]
)

In [ ]:
# モデルを読み込み
model = tf.keras.models.load_model('model_dense.keras', custom_objects={'binary_crossentropy_balance': binary_crossentropy_balance})

In [ ]:
# テストデータの予測
y_pred = model.predict(
    generator(test_datasets, 
              is_augmentation=False,
              is_shuffle=False, 
              is_one_epoch=True,
              batch_size=1,
              ),
    steps=len(test_datasets)
)

In [ ]:
print(f'y_pred shape: {y_pred.shape}')
print(f'test_datasets shape: {test_datasets.shape}')

In [ ]:
# 提出用のデータフレームを作成
submission = pd.DataFrame()
submission["isic_id"] = test_datasets.reshape(-1)
submission["target"] = y_pred
submission.to_csv("submission.csv", index=False)

# 正解用のデータフレームを作成
solution = pd.DataFrame()
solution["isic_id"] = test_datasets
# float に変換して保存
solution["target"] = df_train[df_train["isic_id"].isin(test_datasets)]["target"].values.astype(float)

In [ ]:
# それぞれの列の型を調べる
print(submission.dtypes)
print(solution.dtypes)

In [ ]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [ ]:
# スコアを計算
score = comp_score(solution["target"], submission["target"], "")
print(f"Score: {score:.4f}")

In [ ]:
# accuracy を計算 (0.5 以上の場合は 1, それ以外は 0)
submission_binary = submission.copy()
submission_binary["target"] = (submission["target"] > 0.5).astype(int)
solution["target"] = solution["target"].astype(int)
accuracy = (submission["target"].round() == solution["target"]).mean()
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
# 結果をいくつか比較
print(submission.head())
print(solution.head())